In [40]:
import glob
import zipfile 
import os
import xml.etree.ElementTree as ET
import csv
import random
from bs4 import BeautifulSoup
import progressbar
import numpy as np
from collections import deque
import json
import collections

In [87]:
paths_common = ['GESTION/TETIER_R4',
                'GESTION/NOJO', # #
                'GESTION/NOM_HTML', #
                'DONNEES/IDENT/NOM',
                'DONNEES/IDENT/PRM',
                'DONNEES/IDENT/ADRESSE',
                'DONNEES/IDENT/CP',
                'DONNEES/IDENT/VILLE',
                'DONNEES/IDENT/MEL',
                'DONNEES/IDENT/TEL',
                'DONNEES/OBJET/OBJET_COMPLET']

give_me_the_tag = ['DONNEES/INFO',
                   'DONNEES/TYPE_ACTIVITE_ORG']

paths_take_all = ['DONNEES/CLASSES/CLASSE']#,
                  #'DONNEES/DESCRIPTEURS',
                  #'DONNEES/CARACTERISTIQUES']


paths_common_2K15 = ['GESTION/REFERENCE/IDWEB',
                     'GESTION/NOM_HTML',
                     'DONNEES/IDENTITE/DENOMINATION',
                     'DONNEES/IDENTITE/CORRESPONDANT',
                     'DONNEES/IDENTITE/ADRESSE',
                     'DONNEES/IDENTITE/CP',
                     'DONNEES/IDENTITE/VILLE',
                     'DONNEES/IDENTITE/MEL',
                     'DONNEES/IDENTITE/TEL',
                     'DONNEES/OBJET/OBJET_COMPLET'
                     'DONNEES/OBJET/CPV/PRINCIPAL']

give_me_the_tag_2K15 = ['GESTION/REFERENCE/TYPE_AVIS/NATURE']

In [42]:
def path2tag(path):
    pavel = ''
    for i in reversed(path):
        if i != '/':
            pavel += i
        if i == '/':
            break
    return pavel[::-1]

In [46]:
def get_tags(year):
    tags = []
    return ['NATURE','IDWEB','NOM_HTML','DENOMINATION','CORRESPONDANT','ADRESSE','CP','VILLE','MEL','TEL','CLASSE','DECISIONS']
    if year in [2005 + i for i in range(10)]:
        
        for path in paths_common:
            tags.append(path2tag(path))
        
        for path in paths_take_all:
            tags.append(path2tag(path))
        
        tags.append('LOTS')
        return tags
    
    elif year in [2015 + i for i in range(4)]:
        
        for path in give_me_the_tag_2K15:
            tags.append(path2tag(path))
            
        for path in paths_common_2K15:
            tags.append(path2tag(path))
        
        tags.append('DECISIONS')
        return tags

In [45]:
get_tags(2015)

['NATURE',
 'IDWEB',
 'NOM_HTML',
 'DENOMINATION',
 'CORRESPONDANT',
 'ADRESSE',
 'CP',
 'VILLE',
 'MEL',
 'TEL',
 'PRINCIPAL',
 'DECISIONS']

In [8]:
def give_me_your_dict(annonce):
    lots = []
    out = {}
    if annonce.find('DONNEES/PROCEDURES/LOTS') is not None:
        reslit = list(annonce.find('DONNEES/PROCEDURES/LOTS').iter())
    else:
        return out
    
    if reslit is not None:
        for element in reslit:
            if element.tag == 'NUM_LOT' and element.text is not None:
                sub_list = []
                for lot_element in reslit[reslit.index(element):]:
                    sub_list.append(lot_element)
                    if reslit.index(lot_element) + 1 < len(reslit):
                        next_element = reslit[reslit.index(lot_element) + 1]
                
                        if next_element.tag == 'NUM_LOT' and next_element.text is not None:
                            lots.append(sub_list)
                            break
    for arr in lots:
        out[lots.index(arr)] = {element.tag: element.text for element in arr}
    
    out = json.dumps(out, ensure_ascii=False)
        
    return out

In [89]:
def xml_parser(paths_common, paths_take_all, file):
    rowsamarr = []
    rows = []
    
    for path in paths_common:
        element = file.find(path)
        if element is not None:
            if element.text is not None:
                rows.append(element.text)
            else:
                rows.append('.')
        elif element is None:
            rows.append('.')
    
    for path in paths_take_all:
        element = file.find(path)
        if element is not None:
            reslist = list(element.iter()) 
            result = ' '.join([element.text for element in reslist if element.text is not None])
            rows.append(result)
        elif element is None:
            rows.append('.')
    
    lots = give_me_your_dict(file)
    if lots is not None:
        rows.append(lots)
    else: 
        rows.append('.')
    return rows


In [31]:
def decision2dic(decision):
    dic = {}
    reslit = list(decision.iter())
    for element in reslit:
        if element.text is not None:
            dic[element.tag] = element.text
    return dic

In [33]:
def xml_parser3000(paths_common, give_me_the_tag, file):
    rows = []
    
    for path in give_me_the_tag:
        rows.append(file.find(path).getchildren()[0].tag)
    
    for path in paths_common:
        element = file.find(path)
        if element is not None:
            if element.text is not None:
                rows.append(element.text)
            else:
                rows.append('.')
        elif element is None:
            rows.append('.')
    
    dic = {}
    k = 0
    decision = file.find('DONNEES/ATTRIBUTION')
    if decision is not None:
        children = decision.getchildren()
        for child in children:
            if child.tag == 'DECISION':
                dic[k] = decision2dic(child)
                k+=1
    dic = json.dumps(dic, ensure_ascii=False)
    rows.append(dic)     
    return rows


In [11]:
def write_to_csv(big_row_list, file_name, year):
    
    with open('{}.csv'.format(file_name), 'w') as file:
        spamwriter = csv.writer(file, delimiter=';', quotechar='"',
                            quoting=csv.QUOTE_ALL)
        spamwriter.writerow(get_tags(year))
        for row in big_row_list:
            spamwriter.writerow(row)
        print('wrote {} rows for year {}'.format(len(big_row_list), year))
         

In [12]:
def pbar(maxlen, text=''):
    
    bar = progressbar.ProgressBar(maxval=maxlen, \
    widgets=[progressbar.Bar('=', '[', ']'), text, progressbar.Percentage(), ' (', progressbar.ETA(), ') '])
    
    bar.start()
    
    return bar

In [13]:
def clean_my_apple(apple):
    
    apple = apple.replace('&apos;', "'")
    apple = apple.replace('&deg;', '°')
    apple = apple.replace('&quot;', '"')
    apple = apple.replace('\t', '')
    apple = apple.replace(',', '')
    apple = apple.replace('\n', '')
    apple = apple.replace('é', 'e')
    apple = apple.replace('ç', 'c')
    apple = apple.replace('è', 'e')
    apple = apple.replace(':', '')
    apple = apple.replace('à', 'a')
    apple = apple.replace('ù', 'u')
    clean_apple = apple.replace('ê', 'e')
    
    return clean_apple 

In [83]:
def write_years2csv(start, end):
    
    years = [start + i for i in range(end - start + 1)]

    
    for i in range(len(years)):
        if years[i] < 2015:
            paths = glob.glob('{}/mp*/*.xml'.format(years[i]))
        #paths = glob.glob('rand/*.xml')
            annonces = []
    
            bar = pbar(len(paths), ' reading {} data'.format(years[i]))
        
    
            for path in paths:
                bar.update(paths.index(path))
                file = open(path, encoding='Latin-1')
                apple = file.read()

                apple = clean_my_apple(apple) #il y a forcément un moyen plus simple

                tree = ET.fromstring(apple)
                a = tree.getchildren()
    
                annonces.append(a)
    
            pavel = []
            bar2 = pbar(len(annonces), ' parsing {} data'.format(years[i]))
            for annonce in annonces:
                bar2.update(annonces.index(annonce))
                for j in range(len(annonce)):
                    extracted_annonce = xml_parser(paths_common, paths_take_all, annonce[j])
                    pavel.append(extracted_annonce)
                
            write_to_csv(pavel, years[i], years[i])
            
        if years[i] > 2014:
            
            paths = glob.glob('{}/BOAMP-*/*.xml'.format(years[i]))
            annonces = []
            pavel = []
            bar4 = pbar(len(paths), ' reading {} data'.format(years[i]))

            for j in range(len(paths)):
                bar4.update(j)
                file = open(paths[j], encoding = 'utf-8')
                
                apple = clean_my_apple(file.read())
                
                tree = ET.fromstring(apple)
                annonces.append(tree)
                
                file.close()
            
            bar5 = pbar(len(annonces), ' parsing {} data'.format(years[i]))

            for j in range(len(annonces)):
                
                bar5.update(j)
                extracted_annonce = xml_parser3000(paths_common_2K15, give_me_the_tag_2K15, annonces[j])
                pavel.append(extracted_annonce)
            
            write_to_csv(pavel, years[i], years[i])


    

In [ ]:
write_years2csv(2005, 2018)

[=                                    ] reading 2005 data  4% (ETA:   0:03:13) 